# Imports

In [1]:
# Basic imports
import pandas as pd
import numpy as np

# Warnings
import warnings 
warnings.simplefilter("ignore")

In [2]:
# Plot
from IPython import display
import seaborn as sns

import matplotlib
import matplotlib.pylab as plt
from jupyterthemes import jtplot

jtplot.style('gruvboxd')
matplotlib.use('nbagg')

# Data Reading

In [3]:
from catboost.datasets import titanic

# Data Reading
df_train, df_test = titanic()
df_train.set_index('PassengerId', inplace=True)
df_test.set_index('PassengerId', inplace=True)

# Split X_train, y_train
target = 'Survived'
features = df_test.columns

y_train = df_train[target]
df_train = df_train[features]

df_train.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Preprocessing

In [4]:
from robusta.preprocessing.category import *
from robusta.preprocessing.numeric import *
from robusta.preprocessing.base import *
from robusta.pipeline import *

nums = ['Age', 'Fare', 'SibSp', 'Parch']
cats = ['Pclass', 'Sex', 'Embarked']

data_prep = FeatureUnion([
        ("numeric", make_pipeline(
            ColumnSelector(nums),
            Imputer(strategy="median"),
            GaussRank(),
            #ColumnRenamer(prefix='gr_'),
        )),
        ("category", make_pipeline(
            ColumnSelector(cats),
            Imputer(strategy="most_frequent"),
            LabelEncoder(),
            #ColumnRenamer(prefix='le_'),
        )),
])

X_train = data_prep.fit_transform(df_train)
X_test = data_prep.transform(df_test)

X_train.head()

,Age,Fare,SibSp,Parch,Pclass,Sex,Embarked
PassengerId,,,,,,,
1,-0.488867,-0.968076,0.600843,-0.214091,2,1,2
2,0.584412,0.853865,0.600843,-0.214091,0,0,0
3,-0.275357,-0.454031,-0.290348,-0.214091,2,0,2
4,0.470091,0.703973,0.600843,-0.214091,0,0,2
5,0.470091,-0.379298,-0.290348,-0.214091,2,1,2


# Fold Preparation

In [5]:
from sklearn.model_selection import KFold
from robusta.resampler import *

encoder = FeatureUnion([
    ('category', make_pipeline(
        ColumnSelector(cats),
        TypeConverter('object'),
        TargetEncoderCV(cv=4).set_params(encoder__smoothing=200.0),
    )),
    ('numeric', make_pipeline(
        ColumnSelector(nums),
        Identity(),
    )),
])

resampler = SMOTE(random_state=50, k_neighbors=30)

fold_pipe = make_pipeline(resampler, encoder)

F_train = fold_pipe.fit_transform(X_train, y_train)

F_train.sample(5, random_state=555)

,Embarked,Pclass,Sex,Age,Fare,SibSp,Parch
928,0.453032,0.672597,0.785270,0.491782,1.040380,0.600843,-0.214091
873,0.457019,0.354773,0.273246,0.888832,-0.271711,-0.290348,-0.214091
293,0.460378,0.346921,0.800238,-0.381808,-0.274934,-0.290348,-0.214091
760,0.457019,0.354773,0.273246,-0.013926,0.016949,-0.290348,-0.214091
126,0.474911,0.326256,0.252564,-0.013926,-0.770943,-0.290348,-0.214091


# Model

In [58]:
%%time
from robusta.model import get_model

model = get_model('LGBM', 'classifier', learning_rate=0.01, n_estimators=300, max_depth=4)
model.fit(X_train, y_train)

CPU times: user 806 ms, sys: 288 ms, total: 1.09 s
Wall time: 612 ms


In [53]:
from sklearn.model_selection import RepeatedStratifiedKFold
from robusta.crossval import *

#cv = 5
cv = RepeatedStratifiedKFold(5, 3, random_state=0)
scoring = ['neg_log_loss', 'accuracy', 'roc_auc']

estimator = make_pipeline(model)
scores = crossval_score(estimator, cv, X_train, y_train, scoring=scoring)
#print('{:.4f} ± {:.4f} [model]'.format(np.mean(scores), np.std(scores)))

estimator = make_pipeline(encoder, model)
scores = crossval_score(estimator, cv, X_train, y_train, scoring=scoring)
#print('{:.4f} ± {:.4f} [resampler + model]'.format(np.mean(scores), np.std(scores)))

estimator = make_pipeline(resampler, model)
scores = crossval_score(estimator, cv, X_train, y_train, scoring=scoring)
#print('{:.4f} ± {:.4f} [encoder + model]'.format(np.mean(scores), np.std(scores)))

[19:25:12] LGBMClassifier
[19:25:13] neg_log_loss: -0.4238 ± 0.0307
[19:25:13] accuracy: 0.8133 ± 0.0205
[19:25:13] roc_auc: 0.8636 ± 0.0271

[19:25:13] LGBMClassifier
[19:25:20] neg_log_loss: -0.4272 ± 0.0337
[19:25:20] accuracy: 0.8152 ± 0.0264
[19:25:20] roc_auc: 0.8608 ± 0.0280

[19:25:20] LGBMClassifier
[19:25:21] neg_log_loss: -0.4302 ± 0.0402
[19:25:21] accuracy: 0.8103 ± 0.0270
[19:25:21] roc_auc: 0.8615 ± 0.0279



In [54]:
estimator = make_pipeline(model)

# Submit

In [55]:
%%time
y_oof, y_sub = crossval_predict(estimator, cv, X_train, y_train, None, X_test, verbose=2)

[19:25:48] LGBMClassifier
[19:25:49] fold_0      train/test: 712/179      score: 0.7765
[19:25:49] fold_1      train/test: 712/179      score: 0.8380
[19:25:49] fold_2      train/test: 713/178      score: 0.8146
[19:25:49] fold_3      train/test: 713/178      score: 0.8427
[19:25:49] fold_4      train/test: 714/177      score: 0.8023
[19:25:49] fold_5      train/test: 712/179      score: 0.8212
[19:25:50] fold_6      train/test: 712/179      score: 0.7821
[19:25:50] fold_7      train/test: 713/178      score: 0.7978
[19:25:50] fold_8      train/test: 713/178      score: 0.8315
[19:25:50] fold_9      train/test: 714/177      score: 0.8192
[19:25:50] fold_10      train/test: 712/179      score: 0.8212
[19:25:50] fold_11      train/test: 712/179      score: 0.8380
[19:25:50] fold_12      train/test: 713/178      score: 0.7921
[19:25:50] fold_13      train/test: 713/178      score: 0.8202
[19:25:51] fold_14      train/test: 714/177      score: 0.8023

[19:25:51] score: 0.8133 ± 0.0205

CPU

In [56]:
y_sub.head(10)

PassengerId
892    0
893    0
894    0
895    0
896    0
897    0
898    1
899    0
900    1
901    0
Name: Survived, dtype: int64

In [57]:
import os

path = os.path.join(os.getcwd(), 'pred')
if not os.path.exists(path):
    os.mkdir(path)

sub_path = os.path.join(path, '0 sub baseline.csv')
oof_path = os.path.join(path, '0 oof baseline.csv')

y_sub.to_csv(sub_path, header=True)
y_oof.to_csv(oof_path, header=True)

### Score:
### `[CV] 0.8133 ± 0.0205`
### `[LB] 0.79425`